In [ ]:
#Library 
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [31]:
df = pd.read_csv('/Users/faustin/Downloads/merged_datasets_with_events.csv',sep =',')

# Deleting the covid period
covid_start = '2020-03-01'
covid_stop = '2022-01-01'
df = df[(df['WORK_DATE'] < covid_start) | (df['WORK_DATE'] > covid_stop)]

df.head()


,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,...,humidity,wind_speed,rain_1h,rain_3h,snow_1h,weather_description,REF_CLOSING_DESCRIPTION,ATTENDANCE,NIGHT_SHOW_FLAG,PARADE_FLAG
0,2018-06-01,2018-06-01 19:00:00,19,2018-06-01 19:15:00.000,Roller Coaster,15,2.0,50.0,75.00,75.00,...,89,1.59,0.22,0.0,0.0,light rain,Open,46804.0,0,0
1,2018-06-01,2018-06-01 20:30:00,20,2018-06-01 20:45:00.000,Zipline,40,12.0,69.0,101.25,101.25,...,93,1.67,0.20,0.0,0.0,light rain,Open,46804.0,0,0
2,2018-06-01,2018-06-01 16:30:00,16,2018-06-01 16:45:00.000,Spinning Coaster,15,5.0,283.0,526.25,438.50,...,87,2.94,0.23,0.0,0.0,light rain,Open,46804.0,0,0
3,2018-06-01,2018-06-01 20:30:00,20,2018-06-01 20:45:00.000,Giant Wheel,30,4.0,0.0,503.75,0.00,...,93,1.67,0.20,0.0,0.0,light rain,Open,46804.0,0,0
4,2018-06-01,2018-06-01 13:45:00,13,2018-06-01 14:00:00.000,Oz Theatre,0,0.0,0.0,0.00,0.00,...,77,1.92,0.31,0.0,0.0,light rain,Fermeture Opérationnelle,46804.0,0,0


In [32]:
# Encoding

#Time management 
df['WORK_DATE'] = pd.to_datetime(df['WORK_DATE'])
df['DEB_TIME'] = pd.to_datetime(df['DEB_TIME'])

df['month'] = df['WORK_DATE'].dt.month
df['day'] = df['WORK_DATE'].dt.day
df['weekday'] = df['WORK_DATE'].dt.dayofweek
df.drop(columns=['WORK_DATE'])

df['hour'] = df['DEB_TIME'].dt.hour
df['minute'] = df['DEB_TIME'].dt.minute
df.drop(columns=['DEB_TIME'])
df.drop(columns=['FIN_TIME'])

#Cyclic function to capture the cyclique evolution of the day 

def encode_cyclic_feature(df, col, max_val):
    df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / max_val)
    df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / max_val)
    df = df.drop(columns=[col])  # Delete the original column 
    return df


df = encode_cyclic_feature(df, 'month', 12) 
df = encode_cyclic_feature(df, 'day', 31)  
df = encode_cyclic_feature(df, 'weekday', 7)  
df = encode_cyclic_feature(df, 'hour', 24)  
df = encode_cyclic_feature(df, 'minute', 60)  

In [33]:
numerical_features = df.select_dtypes(include=['float64']).columns.tolist()
print("Variables numériques :", numerical_features)

Variables numériques : ['NB_UNITS', 'GUEST_CARRIED', 'CAPACITY', 'ADJUST_CAPACITY', 'NB_MAX_UNIT', 'temp', 'feels_like', 'wind_speed', 'rain_1h', 'rain_3h', 'snow_1h', 'ATTENDANCE', 'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos']


In [36]:
#ATTENTIO J'AI DROP L'ATTENDANCE

'''
Colonnes garder : 
feels_like
wind_speed
weather
REF_CLOSING_DESCRIPTION
NIGHT_SHOW_FLAG
PARADE_FLAG
weather_description

'''

df = df.drop(columns=['DEB_TIME_HOUR', 'NB_UNITS', 'GUEST_CARRIED', 'CAPACITY', 
                      'ADJUST_CAPACITY', 'NB_MAX_UNIT', 'temp', 'humidity', 'ATTENDANCE'])


#Attention j'ai supprimé les données de pluie 
df.columns

Index(['WORK_DATE', 'DEB_TIME', 'FIN_TIME', 'ENTITY_DESCRIPTION_SHORT',
       'WAIT_TIME_MAX', 'OPEN_TIME', 'UP_TIME', 'DOWNTIME', 'feels_like',
       'wind_speed', 'rain_1h', 'rain_3h', 'snow_1h', 'weather_description',
       'REF_CLOSING_DESCRIPTION', 'NIGHT_SHOW_FLAG', 'PARADE_FLAG',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'],
      dtype='object')

In [39]:
#Train test split 
np.random.seed(42)
X = df[['ENTITY_DESCRIPTION_SHORT', 'feels_like', 'wind_speed', 'weather_description',
        'REF_CLOSING_DESCRIPTION', 'NIGHT_SHOW_FLAG', 'PARADE_FLAG',
        'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 
        'weekday_cos', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos']]

y = df['WAIT_TIME_MAX']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [ ]:
'''
Paramaters :
  objective=
    n_estimators=
    learning_rate=
    max_depth=
    subsample=
    colsample_bytree=
    random_state=42

'''

seach_space = {
    'n_estimators': []
}


In [ ]:
model = xgb.XGBRegressor(
    
)

(1184750, 17) (1184750,)
